In [1]:
from tqdm.auto import tqdm
import copy

In [2]:
test = False
if test:
    text = """....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#..."""
else:
    with open('06_input.txt', 'r') as file:
        text = file.read()
        
current_map = [list(x) for x in text.split('\n')]

In [3]:
print(len(current_map), len(current_map[0]))

130 130


# Task 1

In [4]:
def get_initial_guard_location(input_map):
    for row_idx in range(len(current_map)):
        row = current_map[row_idx]
        for col_idx in range(len(row)):
            char = row[col_idx]
            if char != '.' and char != '#':
                return char, col_idx, row_idx
            
def get_obstacle_locations():
    locations = set()
    for row_idx in range(len(current_map)):
        row = current_map[row_idx]
        for col_idx in range(len(row)):
            char = row[col_idx]
            if char == '#':
                locations.add((row_idx, col_idx))
    return locations

obstacle_locations = get_obstacle_locations()

In [5]:
def update_rotation(rotation):
    rotation_lut = {'^': '>', '>': 'v', 'v': '<', '<': '^'}
    return rotation_lut[rotation]

def evaluate_bounds(x, y):
    # xmin, xmax, ymin, ymax
    bounds = 0, len(current_map), 0, len(current_map[0])
    return x >= bounds[0] and x < bounds[1] and y >= bounds[0] and y < bounds[1]

# just for debug purposes
def print_map(x, y, rotation):
    updated_map = copy.deepcopy(current_map)
    updated_map[y][x] = rotation
    print('\n'.join([''.join(x) for x in updated_map]))
    
def print_final_map(locations):
    updated_map = copy.deepcopy(current_map)
    for (y, x) in locations:
        updated_map[y][x] = 'X'
    print('\n'.join([''.join(x) for x in updated_map]))

# get the updated location by checking the row_column in the right order until there is either a an obstacle or a wall
# TODO: this is still wrong!
def update_location(input_map, rotation, y, x):
    in_bounds=True
    # adding the initial location of the guard
    visited_locations = {(y, x)}
    char = None
    next_char = None
    while in_bounds:
        while next_char != '#':
            if rotation == '^':
                y -= 1
            elif rotation == '>':
                x += 1
            elif rotation == 'v':
                y += 1
            elif rotation == '<':
                x -= 1
            else:
                raise ValueError(f"Unexpected rotation: {rotation}")
            in_bounds = evaluate_bounds(x, y)
            if not in_bounds:
                #print_final_map(visited_locations)
                print(f"Leaving map at {x=}, {y=}")
                return visited_locations
            if (y, x) not in obstacle_locations:
                visited_locations.add((y, x))
            # print(f"prev: {next_char}, checking {y, x}")
            # print_map(x, y, '?')
            # print_final_map(visited_locations)
            next_char = input_map[y][x]
        rotation = update_rotation(rotation)
        # y, x at this point is the location of the obstacle, so the last operation needs to be reverted
        # at the same time, we need to update the next position to check
        if rotation == '^':
            x += 1
            y -= 1
        elif rotation == '>':
            x += 1
            y += 1
        elif rotation == 'v':
            x -= 1
            y += 1
        elif rotation == '<':
            x -= 1
            y -= 1
        else:
            raise ValueError(f"Unexpected rotation: {rotation}")
        next_char = input_map[y][x]
        in_bounds = evaluate_bounds(x, y)
        if not in_bounds:
            #print_final_map(visited_locations)
            print(f"Leaving map at {x=}, {y=}")
            return visited_locations
        if (y, x) not in obstacle_locations:
            visited_locations.add((y, x))
        # print(f"updating rotation: {rotation}, {y}, {x}")
        # print_map(x, y, '?')
        # print_final_map(visited_locations)
        
#print_final_map(set())
#print("==========")
orientation, x, y = get_initial_guard_location(current_map)
visited_locations = update_location(current_map, orientation, y, x)
len(visited_locations)

Leaving map at x=105, y=-1


5534

# Task 2

This is insanely inefficient and could be optimized a lot, but I don't have the time right now, so slow and dirty it is :P

In [13]:
# we know that the obstacle can only be relevant in a position that is visited by the guard at some point
# we need to update the input map with the new obstacle and then check if the guard exits the lab or stays in bounds
# Q: how to figure out if in loop? Rot and pos need to have been already visited in this combination

def check_if_guard_in_loop(input_map, rotation, y, x):
    # returns 1 if results in loop 0 otherwise
    in_bounds = True
    found_loop = False
    # adding the initial location of the guard
    visited_locations = [(y, x, rotation)]
    char = None
    next_char = None
    while in_bounds:
        while next_char != '#':
            if rotation == '^':
                y -= 1
            elif rotation == '>':
                x += 1
            elif rotation == 'v':
                y += 1
            elif rotation == '<':
                x -= 1
            else:
                raise ValueError(f"Unexpected rotation: {rotation}")
                
            # check for bounds
            in_bounds = evaluate_bounds(x, y)
            if not in_bounds:
                return 0
            
            # check if is obstacle
            if (y, x) not in obstacle_locations:
                # check if already has been visited in this rotation
                if (y, x, rotation) in visited_locations:
                    return 1
                visited_locations.append((y, x, rotation))
            next_char = input_map[y][x]
        rotation = update_rotation(rotation)
        # y, x at this point is the location of the obstacle, so the last operation needs to be reverted
        # at the same time, we need to update the next position to check
        if rotation == '^':
            x += 1
            y -= 1
        elif rotation == '>':
            x += 1
            y += 1
        elif rotation == 'v':
            x -= 1
            y += 1
        elif rotation == '<':
            x -= 1
            y -= 1
        else:
            raise ValueError(f"Unexpected rotation: {rotation}")
        next_char = input_map[y][x]
        
        # check for bounds
        in_bounds = evaluate_bounds(x, y)
        if not in_bounds:
            #print_final_map(visited_locations)
            return 0
        
        # check if is obstacle
        if (y, x) not in obstacle_locations:
            # check if already has been visited in this rotation
            if (y, x, rotation) in visited_locations:
                return 1
            visited_locations.append((y, x, rotation))
        
        
orientation, x, y = get_initial_guard_location(current_map)
res = 0
for position in tqdm(visited_locations):
    map_to_test = copy.deepcopy(current_map)
    map_to_test[position[0]][position[1]] = '#'
    res += check_if_guard_in_loop(map_to_test, orientation, y, x)
res 

  0%|          | 0/5534 [00:00<?, ?it/s]

2262